In [5]:
import json
import numpy as np
import os
import pandas as pd
import pickle

from autoimpute_copy.imputations import listwise_delete
from models import PooledLinearModelFactory
from utils import init_results_struct

def _print_cont_metrics(result_struct):
    mae = result_struct['mae']
    mae_iqr = result_struct['mae_values_iqr']
    mse = result_struct['mse']
    mse_iqr = result_struct['mse_values_iqr']
    r2 = result_struct['r2']
    r2_iqr = result_struct['r2_iqr']
        
    print(f'MAE: {mae:0.3f} +/- {mae_iqr:0.3f}')
    print(f'MSE: {mse:0.3f} +/- {mse_iqr:0.3f}')
    print(f'R2: {r2:0.3f} +/- {r2_iqr:0.3f}')

def run_external_val(das_type, treatment, target_treatment, outcome_col = 'class_poor', _type = 'log'):
    test_data_df = pd.read_csv(f'./data/das28_BIOP_{das_type}_{target_treatment}_outcomes.csv')
    
    imputed_cols = [outcome_col, 'BIO', 'das_tend.0', 'das_vas.0', 'das_swol.0', f'{das_type}.0', 'FIRSTBIO', 'WEIGHT', 'HEIGHT', 'DISDUR', 'AGEONSET', 'HAQ', 'SEX', 'SERO', 'CONCURRENT_DMARD', 'HAD_D', 'HAD_A']
    test_data_df = listwise_delete(test_data_df[imputed_cols]).reset_index(drop = True)
    n = test_data_df.shape[0]
    
    X = test_data_df[imputed_cols[2:]].to_numpy()
    y = test_data_df[outcome_col].to_numpy()
        
    pooled_lin_model = PooledLinearModelFactory.init_from_file(f'./data/results/{treatment}/{outcome_col}/{das_type}_{treatment}_{outcome_col}_pooled_model')
    
    M = 500
    
    results = init_results_struct(_type, M)
    
    N = X.shape[0]
    pop = np.arange(N)
    
    for n_boot in range(M):
        boot_idx = np.random.choice(pop, N)
    
        X_boot = X[boot_idx, :]
        y_boot = y[boot_idx]
        
        results.add_result(n_boot, pooled_lin_model, X_boot, y_boot)
    
    result_struct = results.to_results_struct()
    
    print('---------')
    print(f'Performance for {das_type} - {treatment} - {outcome_col} - {target_treatment}')
    
    if _type == 'cont':
        _print_cont_metrics(result_struct)
    print('---------')
    
    os.makedirs(f'./data/results/{treatment}/{outcome_col}/external_val', exist_ok = True)
    
    pickle.dump(result_struct, open(f"./data/results/{treatment}/{outcome_col}/external_val/external_val_{das_type}_{outcome_col}_{treatment}_{target_treatment}.data", "wb"))

CRP Models:

In [2]:
print('Counter Factual Validation:')
run_external_val('crp', 'etanercept', 'adalimumab')
run_external_val('crp', 'adalimumab', 'etanercept')

Counter Factual Validation:
---------
External Val Performance for crp - etanercept - class_poor - adalimumab
Treatments (273): (array(['adalimumab'], dtype=object), array([273]))
Outcomes (273): (array([0, 1]), array([224,  49]))
AUC: 0.622 +/- 0.042
---------
---------
External Val Performance for crp - adalimumab - class_poor - etanercept
Treatments (473): (array(['etanercept'], dtype=object), array([473]))
Outcomes (473): (array([0, 1]), array([382,  91]))
AUC: 0.580 +/- 0.032
---------


In [3]:
print('External TNFi Validation:')

run_external_val('crp', 'etanercept', 'cert_gol_inflix')
run_external_val('crp', 'adalimumab', 'cert_gol_inflix')
run_external_val('crp', 'eta_ada', 'cert_gol_inflix')

External TNFi Validation:
---------
External Val Performance for crp - etanercept - class_poor - cert_gol_inflix
Treatments (195): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([124,  55,  16]))
Outcomes (195): (array([0, 1]), array([149,  46]))
AUC: 0.638 +/- 0.049
---------
---------
External Val Performance for crp - adalimumab - class_poor - cert_gol_inflix
Treatments (195): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([124,  55,  16]))
Outcomes (195): (array([0, 1]), array([149,  46]))
AUC: 0.607 +/- 0.046
---------
---------
External Val Performance for crp - eta_ada - class_poor - cert_gol_inflix
Treatments (195): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([124,  55,  16]))
Outcomes (195): (array([0, 1]), array([149,  46]))
AUC: 0.654 +/- 0.044
---------


In [4]:
print('External Non TNFi Validation:')

run_external_val('crp', 'etanercept', 'nTNFi')
run_external_val('crp', 'adalimumab', 'nTNFi')
run_external_val('crp', 'eta_ada', 'nTNFi')
run_external_val('crp', 'TNFi', 'nTNFi')

External Non TNFi Validation:
---------
External Val Performance for crp - etanercept - class_poor - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 142]))
Outcomes (414): (array([0, 1]), array([303, 111]))
AUC: 0.620 +/- 0.031
---------
---------
External Val Performance for crp - adalimumab - class_poor - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 142]))
Outcomes (414): (array([0, 1]), array([303, 111]))
AUC: 0.535 +/- 0.031
---------
---------
External Val Performance for crp - eta_ada - class_poor - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 142]))
Outcomes (414): (array([0, 1]), array([303, 111]))
AUC: 0.599 +/- 0.031
---------
---------
External Val Performance for crp - TNFi - class_poor - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 142]

ESR Models:

In [5]:
print('Counter Factual Validation:')

run_external_val('esr', 'etanercept', 'adalimumab')
run_external_val('esr', 'adalimumab', 'etanercept')

Counter Factual Validation:
---------
External Val Performance for esr - etanercept - class_poor - adalimumab
Treatments (226): (array(['adalimumab'], dtype=object), array([226]))
Outcomes (226): (array([0, 1]), array([183,  43]))
AUC: 0.742 +/- 0.043
---------
---------
External Val Performance for esr - adalimumab - class_poor - etanercept
Treatments (421): (array(['etanercept'], dtype=object), array([421]))
Outcomes (421): (array([0, 1]), array([330,  91]))
AUC: 0.619 +/- 0.036
---------


In [6]:
print('External TNFi Validation:')

run_external_val('esr', 'etanercept', 'cert_gol_inflix')
run_external_val('esr', 'adalimumab', 'cert_gol_inflix')
run_external_val('esr', 'eta_ada', 'cert_gol_inflix')

External TNFi Validation:
---------
External Val Performance for esr - etanercept - class_poor - cert_gol_inflix
Treatments (140): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([79, 48, 13]))
Outcomes (140): (array([0, 1]), array([100,  40]))
AUC: 0.665 +/- 0.050
---------
---------
External Val Performance for esr - adalimumab - class_poor - cert_gol_inflix
Treatments (140): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([79, 48, 13]))
Outcomes (140): (array([0, 1]), array([100,  40]))
AUC: 0.601 +/- 0.056
---------
---------
External Val Performance for esr - eta_ada - class_poor - cert_gol_inflix
Treatments (140): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([79, 48, 13]))
Outcomes (140): (array([0, 1]), array([100,  40]))
AUC: 0.640 +/- 0.053
---------


In [7]:
print('External Non TNFi Validation:')

run_external_val('esr', 'etanercept', 'nTNFi')
run_external_val('esr', 'adalimumab', 'nTNFi')
run_external_val('esr', 'eta_ada', 'nTNFi')
run_external_val('esr', 'TNFi', 'nTNFi')

External Non TNFi Validation:
---------
External Val Performance for esr - etanercept - class_poor - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 130]))
Outcomes (359): (array([0, 1]), array([257, 102]))
AUC: 0.620 +/- 0.031
---------
---------
External Val Performance for esr - adalimumab - class_poor - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 130]))
Outcomes (359): (array([0, 1]), array([257, 102]))
AUC: 0.621 +/- 0.032
---------
---------
External Val Performance for esr - eta_ada - class_poor - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 130]))
Outcomes (359): (array([0, 1]), array([257, 102]))
AUC: 0.630 +/- 0.032
---------
---------
External Val Performance for esr - TNFi - class_poor - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 130]

CRP MN Models:

In [8]:
print('Counter Factual Validation:')
run_external_val('crp', 'etanercept', 'adalimumab', outcome_col = 'class', _type = 'mnlog')
run_external_val('crp', 'adalimumab', 'etanercept', outcome_col = 'class', _type = 'mnlog')

Counter Factual Validation:
---------
External Val Performance for crp - etanercept - class - adalimumab
Treatments (273): (array(['adalimumab'], dtype=object), array([273]))
Outcomes (273): (array([0, 1, 2]), array([115, 109,  49]))
AUC: 0.614 +/- 0.025
---------
---------
External Val Performance for crp - adalimumab - class - etanercept
Treatments (473): (array(['etanercept'], dtype=object), array([473]))
Outcomes (473): (array([0, 1, 2]), array([180, 202,  91]))
AUC: 0.584 +/- 0.020
---------


In [9]:
print('External TNFi Validation:')

run_external_val('crp', 'etanercept', 'cert_gol_inflix', outcome_col = 'class', _type = 'mnlog')
run_external_val('crp', 'adalimumab', 'cert_gol_inflix', outcome_col = 'class', _type = 'mnlog')
run_external_val('crp', 'eta_ada', 'cert_gol_inflix', outcome_col = 'class', _type = 'mnlog')

External TNFi Validation:
---------
External Val Performance for crp - etanercept - class - cert_gol_inflix
Treatments (195): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([124,  55,  16]))
Outcomes (195): (array([0, 1, 2]), array([75, 74, 46]))
AUC: 0.644 +/- 0.029
---------
---------
External Val Performance for crp - adalimumab - class - cert_gol_inflix
Treatments (195): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([124,  55,  16]))
Outcomes (195): (array([0, 1, 2]), array([75, 74, 46]))
AUC: 0.587 +/- 0.028
---------
---------
External Val Performance for crp - eta_ada - class - cert_gol_inflix
Treatments (195): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([124,  55,  16]))
Outcomes (195): (array([0, 1, 2]), array([75, 74, 46]))
AUC: 0.638 +/- 0.027
---------


In [10]:
print('External Non TNFi Validation:')

run_external_val('crp', 'etanercept', 'nTNFi', outcome_col = 'class', _type = 'mnlog')
run_external_val('crp', 'adalimumab', 'nTNFi', outcome_col = 'class', _type = 'mnlog')
run_external_val('crp', 'eta_ada', 'nTNFi', outcome_col = 'class', _type = 'mnlog')
run_external_val('crp', 'TNFi', 'nTNFi', outcome_col = 'class', _type = 'mnlog')

External Non TNFi Validation:
---------
External Val Performance for crp - etanercept - class - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 142]))
Outcomes (414): (array([0, 1, 2]), array([119, 184, 111]))
AUC: 0.639 +/- 0.022
---------
---------
External Val Performance for crp - adalimumab - class - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 142]))
Outcomes (414): (array([0, 1, 2]), array([119, 184, 111]))
AUC: 0.578 +/- 0.022
---------
---------
External Val Performance for crp - eta_ada - class - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 142]))
Outcomes (414): (array([0, 1, 2]), array([119, 184, 111]))
AUC: 0.634 +/- 0.022
---------
---------
External Val Performance for crp - TNFi - class - nTNFi
Treatments (414): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 48, 224, 

ESR MN Models:

In [11]:
print('Counter Factual Validation:')

run_external_val('esr', 'etanercept', 'adalimumab', outcome_col = 'class', _type = 'mnlog')
run_external_val('esr', 'adalimumab', 'etanercept', outcome_col = 'class', _type = 'mnlog')

Counter Factual Validation:
---------
External Val Performance for esr - etanercept - class - adalimumab
Treatments (226): (array(['adalimumab'], dtype=object), array([226]))
Outcomes (226): (array([0, 1, 2]), array([ 65, 118,  43]))
AUC: 0.686 +/- 0.029
---------
---------
External Val Performance for esr - adalimumab - class - etanercept
Treatments (421): (array(['etanercept'], dtype=object), array([421]))
Outcomes (421): (array([0, 1, 2]), array([127, 203,  91]))
AUC: 0.608 +/- 0.023
---------


In [12]:
print('External TNFi Validation:')

run_external_val('esr', 'etanercept', 'cert_gol_inflix', outcome_col = 'class', _type = 'mnlog')
run_external_val('esr', 'adalimumab', 'cert_gol_inflix', outcome_col = 'class', _type = 'mnlog')
run_external_val('esr', 'eta_ada', 'cert_gol_inflix', outcome_col = 'class', _type = 'mnlog')

External TNFi Validation:
---------
External Val Performance for esr - etanercept - class - cert_gol_inflix
Treatments (140): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([79, 48, 13]))
Outcomes (140): (array([0, 1, 2]), array([41, 59, 40]))
AUC: 0.673 +/- 0.035
---------
---------
External Val Performance for esr - adalimumab - class - cert_gol_inflix
Treatments (140): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([79, 48, 13]))
Outcomes (140): (array([0, 1, 2]), array([41, 59, 40]))
AUC: 0.633 +/- 0.037
---------
---------
External Val Performance for esr - eta_ada - class - cert_gol_inflix
Treatments (140): (array(['Certolizumab', 'golimumab', 'infliximab'], dtype=object), array([79, 48, 13]))
Outcomes (140): (array([0, 1, 2]), array([41, 59, 40]))
AUC: 0.662 +/- 0.035
---------


In [13]:
print('External Non TNFi Validation:')

run_external_val('esr', 'etanercept', 'nTNFi', outcome_col = 'class', _type = 'mnlog')
run_external_val('esr', 'adalimumab', 'nTNFi', outcome_col = 'class', _type = 'mnlog')
run_external_val('esr', 'eta_ada', 'nTNFi', outcome_col = 'class', _type = 'mnlog')
run_external_val('esr', 'TNFi', 'nTNFi', outcome_col = 'class', _type = 'mnlog')

External Non TNFi Validation:
---------
External Val Performance for esr - etanercept - class - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 130]))
Outcomes (359): (array([0, 1, 2]), array([ 75, 182, 102]))
AUC: 0.622 +/- 0.024
---------
---------
External Val Performance for esr - adalimumab - class - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 130]))
Outcomes (359): (array([0, 1, 2]), array([ 75, 182, 102]))
AUC: 0.618 +/- 0.023
---------
---------
External Val Performance for esr - eta_ada - class - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 130]))
Outcomes (359): (array([0, 1, 2]), array([ 75, 182, 102]))
AUC: 0.629 +/- 0.026
---------
---------
External Val Performance for esr - TNFi - class - nTNFi
Treatments (359): (array(['abatacept', 'rituximab', 'tocilizumab'], dtype=object), array([ 50, 179, 

2C DAS Models

In [6]:
run_external_val('crp', 'etanercept', 'adalimumab', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'adalimumab', 'etanercept', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'etanercept', 'TNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'adalimumab', 'TNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'eta_ada', 'TNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'etanercept', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'adalimumab', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'eta_ada', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('crp', 'TNFi', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')

---------
Performance for crp - etanercept - 2cdas_fu
MAE: 2.904 +/- 0.203
MSE: 14.295 +/- 2.062
R2: 0.151 +/- 0.060
---------
---------
Performance for crp - adalimumab - 2cdas_fu
MAE: 3.059 +/- 0.161
MSE: 16.583 +/- 1.715
R2: 0.101 +/- 0.053
---------
---------
Performance for crp - etanercept - 2cdas_fu
MAE: 2.941 +/- 0.113
MSE: 14.826 +/- 1.234
R2: 0.181 +/- 0.030
---------
---------
Performance for crp - adalimumab - 2cdas_fu
MAE: 2.946 +/- 0.122
MSE: 15.556 +/- 1.333
R2: 0.141 +/- 0.035
---------
---------
Performance for crp - eta_ada - 2cdas_fu
MAE: 2.910 +/- 0.116
MSE: 14.770 +/- 1.243
R2: 0.186 +/- 0.028
---------
---------
Performance for crp - etanercept - 2cdas_fu
MAE: 3.089 +/- 0.156
MSE: 15.520 +/- 1.531
R2: 0.085 +/- 0.061
---------
---------
Performance for crp - adalimumab - 2cdas_fu
MAE: 3.349 +/- 0.222
MSE: 19.629 +/- 3.829
R2: -0.149 +/- 0.213
---------
---------
Performance for crp - eta_ada - 2cdas_fu
MAE: 3.148 +/- 0.173
MSE: 16.144 +/- 1.746
R2: 0.052 +/- 0.088

In [7]:
run_external_val('esr', 'etanercept', 'adalimumab', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'adalimumab', 'etanercept', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'etanercept', 'TNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'adalimumab', 'TNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'eta_ada', 'TNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'etanercept', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'adalimumab', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'eta_ada', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')
run_external_val('esr', 'TNFi', 'nTNFi', outcome_col = '2cdas_fu', _type = 'cont')

---------
Performance for esr - etanercept - 2cdas_fu
MAE: 1.521 +/- 0.119
MSE: 4.004 +/- 0.721
R2: 0.204 +/- 0.063
---------
---------
Performance for esr - adalimumab - 2cdas_fu
MAE: 1.705 +/- 0.095
MSE: 4.941 +/- 0.604
R2: 0.080 +/- 0.057
---------
---------
Performance for esr - etanercept - 2cdas_fu
MAE: 1.596 +/- 0.065
MSE: 4.511 +/- 0.389
R2: 0.185 +/- 0.032
---------
---------
Performance for esr - adalimumab - 2cdas_fu
MAE: 1.654 +/- 0.067
MSE: 4.792 +/- 0.433
R2: 0.138 +/- 0.039
---------
---------
Performance for esr - eta_ada - 2cdas_fu
MAE: 1.588 +/- 0.073
MSE: 4.464 +/- 0.409
R2: 0.195 +/- 0.032
---------
---------
Performance for esr - etanercept - 2cdas_fu
MAE: 1.910 +/- 0.105
MSE: 5.893 +/- 0.609
R2: 0.009 +/- 0.072
---------
---------
Performance for esr - adalimumab - 2cdas_fu
MAE: 2.018 +/- 0.121
MSE: 6.870 +/- 1.314
R2: -0.157 +/- 0.219
---------
---------
Performance for esr - eta_ada - 2cdas_fu
MAE: 1.916 +/- 0.109
MSE: 5.992 +/- 0.706
R2: -0.007 +/- 0.088
------

4C DAS Models

In [6]:
run_external_val('crp', 'etanercept', 'adalimumab', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'adalimumab', 'etanercept', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'etanercept', 'TNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'adalimumab', 'TNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'eta_ada', 'TNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'etanercept', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'adalimumab', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'eta_ada', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('crp', 'TNFi', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')

---------
Performance for crp - etanercept - 4cdas_fu - adalimumab
MAE: 0.998 +/- 0.062
MSE: 1.562 +/- 0.182
R2: 0.130 +/- 0.063
---------
---------
Performance for crp - adalimumab - 4cdas_fu - etanercept
MAE: 1.094 +/- 0.043
MSE: 1.745 +/- 0.129
R2: 0.074 +/- 0.053
---------
---------
Performance for crp - etanercept - 4cdas_fu - TNFi
MAE: 1.021 +/- 0.031
MSE: 1.583 +/- 0.092
R2: 0.152 +/- 0.028
---------
---------
Performance for crp - adalimumab - 4cdas_fu - TNFi
MAE: 1.039 +/- 0.032
MSE: 1.616 +/- 0.088
R2: 0.127 +/- 0.036
---------
---------
Performance for crp - eta_ada - 4cdas_fu - TNFi
MAE: 1.022 +/- 0.029
MSE: 1.573 +/- 0.088
R2: 0.160 +/- 0.028
---------
---------
Performance for crp - etanercept - 4cdas_fu - nTNFi
MAE: 1.054 +/- 0.046
MSE: 1.713 +/- 0.123
R2: 0.068 +/- 0.051
---------
---------
Performance for crp - adalimumab - 4cdas_fu - nTNFi
MAE: 1.106 +/- 0.061
MSE: 2.009 +/- 0.319
R2: -0.082 +/- 0.156
---------
---------
Performance for crp - eta_ada - 4cdas_fu - nTNF

In [7]:
run_external_val('esr', 'etanercept', 'adalimumab', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'adalimumab', 'etanercept', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'etanercept', 'TNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'adalimumab', 'TNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'eta_ada', 'TNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'etanercept', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'adalimumab', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'eta_ada', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')
run_external_val('esr', 'TNFi', 'nTNFi', outcome_col = '4cdas_fu', _type = 'cont')

---------
Performance for esr - etanercept - 4cdas_fu - adalimumab
MAE: 0.878 +/- 0.060
MSE: 1.250 +/- 0.150
R2: 0.185 +/- 0.058
---------
---------
Performance for esr - adalimumab - 4cdas_fu - etanercept
MAE: 1.052 +/- 0.045
MSE: 1.607 +/- 0.127
R2: 0.057 +/- 0.062
---------
---------
Performance for esr - etanercept - 4cdas_fu - TNFi
MAE: 0.963 +/- 0.030
MSE: 1.413 +/- 0.088
R2: 0.166 +/- 0.030
---------
---------
Performance for esr - adalimumab - 4cdas_fu - TNFi
MAE: 0.994 +/- 0.036
MSE: 1.490 +/- 0.105
R2: 0.122 +/- 0.040
---------
---------
Performance for esr - eta_ada - 4cdas_fu - TNFi
MAE: 0.963 +/- 0.035
MSE: 1.408 +/- 0.094
R2: 0.171 +/- 0.030
---------
---------
Performance for esr - etanercept - 4cdas_fu - nTNFi
MAE: 1.081 +/- 0.050
MSE: 1.749 +/- 0.157
R2: 0.019 +/- 0.062
---------
---------
Performance for esr - adalimumab - 4cdas_fu - nTNFi
MAE: 1.077 +/- 0.060
MSE: 1.866 +/- 0.257
R2: -0.041 +/- 0.123
---------
---------
Performance for esr - eta_ada - 4cdas_fu - nTNF